In [17]:
import pandas as pd
import numpy as np
import copy as cp

import statsmodels.api as sm
import sklearn.metrics as metrics

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
onedrive_path = "C:/Users/cfowle/The Estée Lauder Companies Inc/TeamAnis - General/"

In [3]:
repeat_rate = pd.read_csv(onedrive_path + "Data/Repeat Rate/combined.csv")
ratings = pd.read_csv(onedrive_path + "Data/Ratings and Reviews/reviews_demand_itemid4.csv")
cc = pd.read_csv(onedrive_path + "Data/Consumer Care/cc_product_month.csv")

In [4]:
cc = cc.rename(columns = {"P4": "itemid_4",
                         "Date Month": "date"})

## Summarize Ratings at FY Level

In [5]:
#ratings = ratings.merge(cc, how = "left")

In [6]:
ratings.date = pd.to_datetime(ratings.date)

In [7]:
FY18_MIN = pd.to_datetime("7/1/2018")
FY19_MIN = pd.to_datetime("7/1/2019")
FY19_MAX = pd.to_datetime("7/1/2020")

ratings["fy"] = [int(18) if x >= FY18_MIN and x < FY19_MIN else int(19) if x >= FY19_MIN and x < FY19_MAX else 0 for x in ratings.date]

In [8]:
ratings = ratings.groupby(["itemid_4", "fy"]).sum().reset_index()

In [9]:
repeat_rate.loc[repeat_rate.brand == "Jo Malone London"]


,fy,brand,itemid_4,category,subcategory,description,transaction_count,percent_brand_mix,product_repeat,brand_repeat,purchase_cycle_days,retail_price,retail_transactions,online_transactions,percent_retail_transactions,percent_online_transactions
321,18,Jo Malone London,6902,FRAGRANCE,WILD BLUEBELL,WILD BLUEBELL COLOGNE 30ML/SLI,8263,0.037636,0.197964,0.364065,82.753508,65.005836,624,7639,0.075517,0.924483
322,18,Jo Malone London,6902,FRAGRANCE,WOOD SAGE AND SEA SALT,WOOD SAGE AND SEA SALT 30ML,7443,0.033901,0.077228,0.268563,93.365063,65.022058,1386,6057,0.186215,0.813785
323,18,Jo Malone London,6902,FRAGRANCE,ENGLISH PEAR & FREESIA,ENGLISH PEAR & FREESIA CLGNE 3,6846,0.031182,0.088330,0.296050,86.046518,65.003569,1160,5686,0.169442,0.830558
324,18,Jo Malone London,6902,SKINCARE,VITAMIN E LIP CONDITIONER,VITAMIN E LIP CONDITIONER,5278,0.024040,0.193330,0.422838,55.167446,29.998124,342,4936,0.064797,0.935203
325,18,Jo Malone London,6902,FRAGRANCE,SOAP,EP&F SOAP,5047,0.022988,0.208962,0.490683,74.028390,21.899528,341,4706,0.067565,0.932435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,19,Jo Malone London,6900,FRAGRANCE,ENGLISH PEAR & FREESIA SET,ENGLISHPEAR&FREESIASET,1,0.000004,0.000000,0.000000,0.000000,135.000000,1,0,1.000000,0.000000
2623,19,Jo Malone London,6900,FRAGRANCE,BODY & HAND WASH,P&BS BODY & HAND WASH/NEW ILN,1,0.000004,0.000000,1.000000,0.000000,40.000000,0,1,0.000000,1.000000
2624,19,Jo Malone London,6900,FRAGRANCE,BODY & HAND WASH,OB BODY & HAND WASH 250ML,1,0.000004,0.000000,0.000000,0.000000,40.000000,1,0,1.000000,0.000000
2625,19,Jo Malone London,6900,FRAGRANCE,BATH OIL,POMEGRANATE NOIR BATH OIL,1,0.000004,0.000000,0.000000,0.000000,65.000000,1,0,1.000000,0.000000


## Join with Repeat Rate

In [10]:
merged = repeat_rate.merge(ratings, on =["itemid_4", "fy"])

In [11]:
merged = merged.loc[merged.demand > 0]
merged = merged.loc[merged.nb_reviews > 10]

In [12]:
merged.columns

Index(['fy', 'brand', 'itemid_4', 'category', 'subcategory', 'description',
       'transaction_count', 'percent_brand_mix', 'product_repeat',
       'brand_repeat', 'purchase_cycle_days', 'retail_price',
       'retail_transactions', 'online_transactions',
       'percent_retail_transactions', 'percent_online_transactions',
       'avg_nb_statements', 'nb_reviews', 'rating', 'rating_1', 'rating_2',
       'rating_3', 'rating_4', 'rating_5', 'sentiment_negative',
       'sentiment_neutral', 'sentiment_positive', 'sentiment', 'demand',
       'demand_1', 'demand_2', 'demand_3'],
      dtype='object')

## Predict Repeat Rates

### Brand Repeat Rate

In [65]:
X = merged[['avg_nb_statements', 'nb_reviews', 'rating', 'rating_1', 'rating_2',
       'rating_3', 'rating_4', 'sentiment_negative',
       'sentiment_neutral', 'sentiment_positive', 'sentiment', 'demand']]
y = merged["brand_repeat"]

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [67]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-4.718298631937919

In [68]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:           brand_repeat   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     1.523
Date:                Thu, 23 Jul 2020   Prob (F-statistic):              0.131
Time:                        10:32:54   Log-Likelihood:                 127.19
No. Observations:                 136   AIC:                            -230.4
Df Residuals:                     124   BIC:                            -195.4
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.3590      0

### Product Repeat Rate

In [69]:
X = merged[['avg_nb_statements', 'nb_reviews', 'rating', 'rating_1', 'rating_2',
       'rating_3', 'rating_4', 'sentiment_negative',
       'sentiment_neutral', 'sentiment_positive', 'sentiment', 'demand']]
y = merged["product_repeat"]

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [71]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-3.2479018531629977

In [72]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:         product_repeat   R-squared:                       0.237
Model:                            OLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                     3.494
Date:                Thu, 23 Jul 2020   Prob (F-statistic):           0.000282
Time:                        10:33:24   Log-Likelihood:                 225.22
No. Observations:                 136   AIC:                            -426.4
Df Residuals:                     124   BIC:                            -391.5
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0836      0

## Try removing the demand variable
### Brand Repeat

In [74]:
X = merged[['avg_nb_statements', 'nb_reviews', 'rating', 'rating_1', 'rating_2',
       'rating_3', 'rating_4', 'sentiment_negative',
       'sentiment_neutral', 'sentiment_positive', 'sentiment']]
y = merged["brand_repeat"]

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [76]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-7.428265447234196

In [77]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:           brand_repeat   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                    0.8052
Date:                Thu, 23 Jul 2020   Prob (F-statistic):              0.624
Time:                        10:33:36   Log-Likelihood:                 131.57
No. Observations:                 136   AIC:                            -241.1
Df Residuals:                     125   BIC:                            -209.1
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.3864      0

Average number of statements is showing significance which is interesting as we know that that is corellatied with negative reviews

### Product Repeat

In [78]:
X = merged[['avg_nb_statements', 'nb_reviews', 'rating', 'rating_1', 'rating_2',
       'rating_3', 'rating_4', 'sentiment_negative',
       'sentiment_neutral', 'sentiment_positive', 'sentiment']]
y = merged["product_repeat"]

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [80]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-56.12482884022123

In [81]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:         product_repeat   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     2.395
Date:                Thu, 23 Jul 2020   Prob (F-statistic):             0.0123
Time:                        10:33:46   Log-Likelihood:                 229.56
No. Observations:                 136   AIC:                            -437.1
Df Residuals:                     125   BIC:                            -405.1
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0773      0

For the product repeat rating three is starting to show something. Let's look at sentiment because there's a crazy amount of multicolinearity going on here

## Sentiment
### Brand Repeat Rate

In [82]:
X = merged[['avg_nb_statements','sentiment_negative',
       'sentiment_neutral', 'sentiment_positive']]
y = merged["brand_repeat"]

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [84]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-20.960400566694513

In [85]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:           brand_repeat   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.8013
Date:                Thu, 23 Jul 2020   Prob (F-statistic):              0.526
Time:                        10:33:55   Log-Likelihood:                 130.21
No. Observations:                 136   AIC:                            -250.4
Df Residuals:                     131   BIC:                            -235.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.3826      0

Nada

### Product Repeat Rate

In [86]:
X = merged[['avg_nb_statements','sentiment_negative',
       'sentiment_neutral', 'sentiment_positive']]
y = merged["product_repeat"]

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [88]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-18.36543439111686

In [89]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:         product_repeat   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.456
Date:                Thu, 23 Jul 2020   Prob (F-statistic):              0.219
Time:                        10:34:06   Log-Likelihood:                 217.87
No. Observations:                 136   AIC:                            -425.7
Df Residuals:                     131   BIC:                            -411.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0873      0

Some small effect from sentiment negative, but the sign is wrong. Let's try logging the right hand side of the regression

## Logged Features
### Brand Repeat

In [90]:
X = np.log(merged[['avg_nb_statements','sentiment_negative',
       'sentiment_neutral', 'sentiment_positive']] + 1)
y = merged["brand_repeat"]

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [92]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-112.38797019505233

In [93]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:           brand_repeat   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                    0.6168
Date:                Thu, 23 Jul 2020   Prob (F-statistic):              0.651
Time:                        10:34:37   Log-Likelihood:                 148.97
No. Observations:                 136   AIC:                            -287.9
Df Residuals:                     131   BIC:                            -273.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.4382      0

Ummmm...nope

### Product Repeat

In [94]:
X = np.log(merged[['avg_nb_statements','sentiment_negative',
       'sentiment_neutral', 'sentiment_positive']] + 1)
y = merged["product_repeat"]

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [96]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
metrics.r2_score(pred, y_test)

-26.781944514775276

In [97]:
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()

print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:         product_repeat   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.177
Date:                Thu, 23 Jul 2020   Prob (F-statistic):              0.324
Time:                        10:34:42   Log-Likelihood:                 215.37
No. Observations:                 136   AIC:                            -420.7
Df Residuals:                     131   BIC:                            -406.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0901      0

Sentiemnt negative keeps on having a positive coefficent, but I'm not sure what to think of it...the R^2 is crazy low, so I don't want to put stock in that